In [13]:
##
#Use Tweepy API (search_recent_tweets) to fetch tweets for the period 6/29-7/6. 
#Limitation with this API is, it can fetch only a week's tweets
#I had no access to GEO tags, so was not able to fetch GEO coordinates
# Ran these codes for Hashtags - [#nftlaunchpqd,#AlgorandNFTs,#CELONFTs,#PolygonNFTs,#NEARNFTs]
##
import tweepy
import time
import pandas as pd

client = tweepy.Client(bearer_token='dd', wait_on_rate_limit=True)

genadrop_tweets = []
paginator = tweepy.Paginator(client.search_recent_tweets, 
                                 query = '#NEARNFTs -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                              max_results=100,limit = 10)

for page in paginator:
    genadrop_tweets.append(page)

In [14]:
##
# Fetch relevant info from Response and frame DATAFRAME
#Fetch user collection from response and retrieve user-specific info
#Save the dataframe to csv file - Running these codes for each hashtags will give five csv files
##
result = []
user_dict = {}

for response in genadrop_tweets:   
    if(response.data != None):
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                  'followers': user.public_metrics['followers_count'],
                                  'tweets': user.public_metrics['tweet_count'],
                                  'description': user.description,
                                  'location': user.location
                                 }
        for tweet in response.data:        
            author_info = user_dict[tweet.author_id] 
            tweet_text = tweet.text.strip().encode('ascii','ignore')
            tweet_text = b''.join(tweet_text.splitlines())
            
            result.append({'author_id': tweet.author_id, 
                           'username': author_info['username'],
                           'author_followers': author_info['followers'],
                           'author_tweets': author_info['tweets'],
                           'author_description': author_info['description'],
                           'author_location': author_info['location'],
                           'text': tweet_text,
                           'created_at': tweet.created_at,
                           'retweets': tweet.public_metrics['retweet_count'],
                           'replies': tweet.public_metrics['reply_count'],
                           'likes': tweet.public_metrics['like_count'],
                           'quote_count': tweet.public_metrics['quote_count']
                          })


        df = pd.DataFrame(result)
        df.to_csv('genadroptweets_1_NEARNFTs_2.csv', encoding='utf-8', index=False)

In [15]:
##
# Combine all five csv files and save as one.
##
import os
from glob import glob
import pandas as pd

direc = "C:\\WWWWW\\NFTs"
searchterm = "Tweepy"
csv_files = glob(os.path.join(direc, searchterm, '*.csv'))

data = []
for csv in csv_files:
    frame = pd.read_csv(csv)
    frame['filename'] = os.path.basename(csv)
    data.append(frame)

tweets_df = pd.concat(data).reset_index(drop = True)
tweets_df.shape

(2410, 13)

In [16]:
tweets_df.to_csv('NFTTweets_Week.csv')